In [3]:
# @title Project Chronos: Final Path & Threading Fix
# Strategy: Change Working Directory (setwd) to avoid path duplication errors.

# --- PHASE 0: SYSTEM SAFETY ---
# 1. Force Single-Threaded Mode (Crucial for Colab Stability)
Sys.setenv(OMP_NUM_THREADS = "1")
Sys.setenv(MKL_NUM_THREADS = "1")
Sys.setenv(OPENBLAS_NUM_THREADS = "1")

# 2. Load Libraries
if (!require("BiocManager", quietly = TRUE)) install.packages("BiocManager")
# Ensure preprocessCore is installed (skip if already present to save time)
if (!require("preprocessCore", quietly = TRUE)) {
    BiocManager::install("preprocessCore", configure.args = "--disable-threading", update = FALSE, ask = FALSE)
}
BiocManager::install(c("affy", "ath1121501cdf"), update = FALSE, ask = FALSE)

library(affy)
library(ath1121501cdf)

# --- PHASE 1: DIRECTORY SWITCH ---
print(paste("[", format(Sys.time(), "%H:%M:%S"), "] 📂 Setting up Environment...", sep=""))

# Define the data folder
data_dir <- "/content/raw_cel"

# Check if directory exists
if (!dir.exists(data_dir)) {
    stop("❌ Directory '/content/raw_cel' not found. Please Extract your .tar files first.")
}

# SAVE current directory (to go back later if needed)
old_wd <- getwd()

# SWITCH directory to the data folder
# This is the MAGIC FIX. It prevents "/content//content/" errors.
setwd(data_dir)
print(paste("   Working Directory changed to:", getwd()))

# --- PHASE 2: PROCESSING ---
print(paste("[", format(Sys.time(), "%H:%M:%S"), "] ⚛️ Running Normalization...", sep=""))

# 1. List files (Now we just get simple filenames like "file.cel")
cel_files <- list.files(pattern = "\\.cel", ignore.case = TRUE, recursive = TRUE)
print(paste("   ✅ Found", length(cel_files), "files."))

# 2. Run justRMA
# Since we are IN the folder, we don't need full paths.
eset <- justRMA(filenames = cel_files, verbose = TRUE)

# --- PHASE 3: EXPORT ---
print(paste("[", format(Sys.time(), "%H:%M:%S"), "] 💾 Saving Matrix...", sep=""))

# Return to original directory to save output
setwd(old_wd)

# Extract Data
gene_matrix <- exprs(eset)

# Clean Names
colnames(gene_matrix) <- gsub("\\.cel|\\.CEL|\\.gz", "", basename(colnames(gene_matrix)))

# Save
output_file <- "temporal_expression_matrix_GENE.csv"
write.csv(gene_matrix, file = output_file, quote = FALSE)

print("✅ SUCCESS!")
print(paste("   Matrix Shape:", nrow(gene_matrix), "Genes x", ncol(gene_matrix), "Samples"))
print(paste("   💾 Saved to:", file.path(getwd(), output_file)))

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.22 (BiocManager 1.30.27), R 4.5.2 (2025-10-31)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'affy' 'ath1121501cdf'”


[1] "[07:30:50] 📂 Setting up Environment..."
[1] "   Working Directory changed to: /content/raw_cel"
[1] "[07:30:50] ⚛️ Running Normalization..."
[1] "   ✅ Found 28 files."
Background correcting
Normalizing
Calculating Expression
[1] "[07:30:58] 💾 Saving Matrix..."
[1] "✅ SUCCESS!"
[1] "   Matrix Shape: 22810 Genes x 28 Samples"
[1] "   💾 Saved to: /content/temporal_expression_matrix_GENE.csv"


In [6]:
# @title Project Chronos: Phase 4 - The Official ID Translation (Fixed)
# Fixes 'Invalid columns: ACCNUM' error by detecting the correct column.

print(paste("[", format(Sys.time(), "%H:%M:%S"), "] 📚 Loading Annotation Database...", sep=""))

if (!require("ath1121501.db", quietly = TRUE)) {
    BiocManager::install("ath1121501.db", update = FALSE, ask = FALSE)
}
library(ath1121501.db)

# --- THE FIX: DETECT CORRECT COLUMN ---
valid_cols <- columns(ath1121501.db)
target_col <- "ACCNUM" # Default

if ("SYMBOL" %in% valid_cols) {
    target_col <- "SYMBOL" # Modern packages use SYMBOL for AGI codes
} else if ("TAIR" %in% valid_cols) {
    target_col <- "TAIR"
}

print(paste("   ✅ Using ID Column:", target_col))

# --- MAP PROBES ---
print("   ... Mapping Probe IDs to Gene Locus IDs...")
probes <- rownames(gene_matrix)

# Map using the detected column
gene_ids <- mapIds(ath1121501.db,
                   keys = probes,
                   column = target_col,
                   keytype = "PROBEID",
                   multiVals = "first")

# --- FILTER & AGGREGATE ---
valid_mask <- !is.na(gene_ids)
clean_matrix <- gene_matrix[valid_mask, ]
clean_genes <- gene_ids[valid_mask]

print(paste("   Probes matching Genes:", sum(valid_mask)))

print("   ... Aggregating signals (Averaging duplicates)...")
final_gene_matrix <- aggregate(clean_matrix, by = list(GeneID = clean_genes), FUN = mean)

# Fix formatting
rownames(final_gene_matrix) <- final_gene_matrix$GeneID
final_gene_matrix$GeneID <- NULL

# --- SAVE FINAL ---
output_filename <- "temporal_expression_matrix_FINAL.csv"
write.csv(final_gene_matrix, file = output_filename, quote = FALSE)

print("✅ TRANSLATION COMPLETE.")
print(paste("   Final Dimensions:", nrow(final_gene_matrix), "Unique Genes x", ncol(final_gene_matrix), "Samples"))
print(paste("   💾 Download this file:", output_filename))

[1] "[07:42:36] 📚 Loading Annotation Database..."
[1] "   ✅ Using ID Column: SYMBOL"
[1] "   ... Mapping Probe IDs to Gene Locus IDs..."


'select()' returned 1:many mapping between keys and columns



[1] "   Probes matching Genes: 13223"
[1] "   ... Aggregating signals (Averaging duplicates)..."
[1] "✅ TRANSLATION COMPLETE."
[1] "   Final Dimensions: 12853 Unique Genes x 28 Samples"
[1] "   💾 Download this file: temporal_expression_matrix_FINAL.csv"
